In [ ]:
checkp = 'checkpoints/tpe'


In [ ]:
import os
import csv, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


epochs = 150*5
#image_dir = os.getcwd() + dir_ + '/checkpoints/%s'%(checkp)
image_dir = os.getcwd() + '/%s'%(checkp)

if('S7' in checkp):

    if('ratio' in checkp): 
        ratio = float((checkp.split('ratio')[-1]).split('_')[0])/100
        epochs = int(epochs / ratio )
    elif('ep' in checkp):
        epochs = int( ( checkp.split('ep')[-1] ).split('_')[0] )
    elif('epoch' in checkp):
        epochs = int( ( checkp.split('epoch')[-1] ).split('_')[0] )
        

trials = [] ; times = []
numpy_acc = [] ; best_acc = []
lrs = []; wds = []; mns = []; pruned = []; 
if('uda' in image_dir): advws = []
elif('MDC' in image_dir): arts = [] ; clips = []; prods = []; rws = []

is_random = ('random' in checkp) ; is_grid = ('grid' in checkp)

for (imagePath, dir, files) in sorted(os.walk(image_dir)):
    csv = sorted(glob.glob(imagePath + "/*.csv"))
    if len(csv) :
        
        # Configuration Setting
        target_dir = imagePath.split('/')[-1]
        trial = int(target_dir.split('_')[0])
        if is_grid:
            lr = float(target_dir.split('lr=')[1].split('_')[0])
        else:
            lr = float(target_dir.split('lr=')[1].split(',')[0])
            wd = float(target_dir.split('wd=')[1].split('_')[0])
            mn = float(target_dir.split('mn=')[1].split(',')[0])
            
            if('uda' in image_dir): 
                advw = float(target_dir.split('advw=')[1].split(',')[0])
            elif('MDC' in image_dir): 
                art = float(target_dir.split('Art=')[1].split(',')[0])
                clip = float(target_dir.split('Clipart=')[1].split(',')[0])
                prod = float(target_dir.split('Product=')[1].split(',')[0])
                rw = float(target_dir.split('RealWorld=')[1].split(',')[0])
                

        try:
            # Configuration Gathering
            df = pd.read_csv(csv[0])
        except:
            pass
        
        if('acc' in df.keys())and('best_acc' in df.keys()):
            trials.append(trial) ; lrs.append(lr)
            best_acc.append(df['best_acc'].max())

            if not(is_grid):
                wds.append(wd) ; mns.append(mn)
                if('uda' in image_dir): 
                    advws.append(advw)
                elif('MDC' in image_dir): 
                    arts.append(art); clips.append(clip)
                    prods.append(prod); rws.append(rw)
            dfacc = df['acc'].to_numpy()
            # For Pruned Trials
            len_acc =len(dfacc)
            if(len_acc!=epochs):
                zeros = np.zeros(epochs-len_acc)
                dfacc = np.concatenate([dfacc,zeros])
                pruned.append(1)
            else: pruned.append(0)
            numpy_acc.append(dfacc)

        time = df['time_since_restore'].max()
        times.append(time)

        
# From list to array        
trs = np.array(trials) ; pr = np.array(pruned)
lrs = np.array(lrs); times = np.array(times)
na = np.array(numpy_acc) ; ba = np.array(best_acc)

# Trail Cutoff
#if(1):
#    trs = trs[:256]; pr = pr[:256]; lrs = lrs[:256]; na = na[:256] ; ba = ba[:256]


sort_idx = np.argsort(trs)
trs = trs[sort_idx]; pr = pr[sort_idx]
lrs = lrs[sort_idx]; times = times[sort_idx]
na = na[sort_idx]; ba = ba[sort_idx]; best_sort = np.argsort(ba)


if not(is_grid):
    wds = np.array(wds);  mns = np.array(mns)
    wds = wds[sort_idx];  mns = mns[sort_idx]

    if('uda' in checkp): 
        advws = np.array(advws) ; advws = advws[sort_idx]
    elif('MDC' in checkp): 
        rws=np.array(rws) ; rws = rws[sort_idx]
        arts=np.array(arts) ; arts = arts[sort_idx]
        clips=np.array(clips) ; clips = clips[sort_idx]
        prods=np.array(prods) ; prods = prods[sort_idx]
        
if('MDC' in checkp): ba = ba * 100 ; na = na * 100

In [ ]:
len(lrs)

In [ ]:
def labeling(topidx):
    if(is_grid): config_label = 'Top%d : %.3f%% (%s %.3f )'%(i+1,ba[topidx],'LR', lrs[topidx])
    else:
        config_label = 'Top%d : %.3f%% (%s %.3f  %s %.5f  %s %.3f  )'%(
            i+1,ba[topidx], 'LR', lrs[topidx],'WD', wds[topidx], 'MN', 1-mns[topidx] )
    return config_label


top = 5

fig = plt.figure(figsize=(36,18)) ## 캔버스 생성
plt.rcParams['xtick.labelsize']=40
plt.rcParams['ytick.labelsize']=40

ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성
idxs = best_sort[-top:]
     
# pink background
for i in range(na.shape[0]):
    ax.plot(na[i],marker='o',color='#f3e5e1') #EFC2B8 #f3e5e1


colors = ['r','g','b','c','y']
if(top>5): colors = colors + ['k'] * (top-5) 

for i in range(top): 
    topidx = idxs[-(i+1)]
    config_label = labeling(topidx)
    ax.plot(na[topidx],marker='o', label= config_label, c=colors[i])
    
#ax.legend() ## 범례 
plt.title('Learning Curves : Top %d Sampling'%(top),fontsize=50) ## 타이틀 설정
plt.legend( prop={'size': 30}, loc='lower right')

if(0):
    #========================================== Ranking ==========================================#
    xs = []
    ratio = 0.4; steps = 10
    for i in range(na.shape[1]):
        if((i+1)%steps==0):        
            rank = []
            temp_sort = np.argsort(na[:,i])
            na_i = na[:,i]
            remains = na_i[na_i!=0]

            [rank.insert(0, na.shape[0] - np.where(idx == temp_sort)[0][0] ) for idx in idxs]
            rank.append( sum(rank) / len(rank) )

            plt.text(i,ratio*i,str(rank[-1])+' / %d'%(len(remains)), fontsize=25, ha='center')
            plt.text(i,ratio*i-3,rank[:-1], fontsize=20, ha='center')

            ax.axvline(i, alpha=0.7)
            xs.append(i+1)
    plt.xticks(xs)

plt.show()

In [ ]:
np.sum(times)/8/3600

In [ ]:
# Procedure
proc = len(trs)
print("Finished Trials :",proc)

# Actual Budget
budget = 0
#cnt = {'1':0, '3':0, '9':0, '27':0,'81':0,'100':0}
cnt = {}
for i in range(len(na)):
    _len = len(np.where(na[i,:]!=0)[0])
    if(str(_len) in cnt.keys()): cnt[str(_len)] += 1
    else: cnt[str(_len)] = 1
    budget += _len

print('Budget : %d epochs'%(budget))

# Average Time
print('Average Time per Trial : %.f sec'%(np.average(times[:proc])))

print('Pruned Trials at each epoch ',cnt)

In [ ]:
# Procedure
proc = len(trs)
print("Finished Trials :",proc)

# Actual Budget
budget = 0
#cnt = {'1':0, '3':0, '9':0, '27':0,'81':0,'100':0}
cnt = {}
for i in range(len(na)):
    _len = len(np.where(na[i,:]!=0)[0])
    if(str(_len) in cnt.keys()): cnt[str(_len)] += 1
    else: cnt[str(_len)] = 1
    budget += _len

print('Budget : %d epochs'%(budget))

# Average Time
print('Average Time per Trial : %.f sec'%(np.average(times[:proc])))

print('Pruned Trials at each epoch ',cnt)

## Sampling Distributions for Hyperparameters

In [ ]:
def sampling(top, hyp, config, setting, log_true):
    xs = np.arange(len(hyp))
    best_trial = np.argsort(ba)[-1]

    plt.figure(figsize=(60,120))
    plt.subplot(611)
    plt.gca().set_title('%s \n \
    Best Trial %d  |  Best Acc %.3f  |  Best Config %.4f'
    %(setting, best_trial, ba[best_trial], hyp[best_trial]),fontdict = {'fontsize' : 70})

    plt.rcParams['xtick.labelsize']=50
    plt.rcParams['ytick.labelsize']=50


    for g in np.unique(pr):
        i = np.where(pr == g)
        if(g==0):
            color='r'
            label='Not Pruned'
        elif(g==1):
            color='#EFC2B8'
            label = 'Pruned'
        plt.scatter(xs[i], hyp[i], label=label, c=color, s=300)


    colors = ['m','b','c','g','y']
    if(top>5): colors = ['k'] * (top-5) + colors

    top_idxs = np.argsort(ba)[-top:]

    for i, idx in enumerate(top_idxs):
        plt.text(xs[idx],hyp[idx],top-i,c='w',fontsize=16, weight='bold', ha='center', ma='center')
        plt.scatter(xs[idx],hyp[idx],label='Top%d(%.3f)'%(top-i,ba[idx]),marker="*", c=colors[i],s=3500)

    if(log_true):plt.gca().set_yscale('log')

    plt.axhline(hyp[top_idxs].min(),c='k')
    plt.axhline(hyp[top_idxs].max(),c='k')
    print('Range :', hyp[top_idxs].min(), hyp[top_idxs].max())
    #plt.ylim([hyp.min()*0.5,hyp.max()*1.1])
    plt.legend( prop={'size': 30})
    plt.xlabel('Trials', fontsize=70)
    plt.ylabel(config, fontsize=70)
    plt.show()

In [ ]:
top = 10 ; log_true = True

hyp = lrs ; config = 'LR' ; setting = 'TPE-LR search'; 
sampling(top, hyp, config, setting, log_true)
if not(is_grid):
    hyp = wds ; config = 'WD' ; setting = 'TPE-WD search'; 
    sampling(top, hyp, config, setting, log_true)

    hyp = mns ; config = 'MN' ; setting = 'TPE-MN search'
    sampling(top, hyp, config, setting, log_true)
    
    if('uda' in checkp):
        hyp = advws ; config = 'AdvW' ; setting = 'TPE-AdvW search'
        sampling(top, hyp, config, setting, log_true)


    if('MDC' in checkp):
        hyp = rws ; config = 'rws' ; setting = 'RealWorld search'
        sampling(top, hyp, config, setting, log_true)
        hyp = arts ; config = 'arts' ; setting = 'Art search'
        sampling(top, hyp, config, setting, log_true)
        hyp = clips ; config = 'clips' ; setting = 'Clipart search'
        sampling(top, hyp, config, setting, log_true)
        hyp = prods ; config = 'prods' ; setting = 'Product search'
        sampling(top, hyp, config, setting, log_true)
        
    

### GIF

In [ ]:
# Create GIF
import os
import imageio

def gif(trs, target):
    images = []
    trials = len(trs)

    for i in range(18,trials-1):
        print('Procedure [%d/%d]'%(i+1,trials), end='\r', flush=True)
        images.append(imageio.imread('%s/indiv_KD_Trial%d_%s.png'%(image_dir,i,target)))

    frames = 4
    imageio.mimsave('%s/indiv_%s_%s_sampling.gif'%(image_dir,image_dir.split('/')[-1],target), images, fps=frames)
    
if not(is_grid) and not(is_random):
    trs = lrs ; target = 'lr' ; gif(trs, target)
#    trs = wds ; target = 'wd' ; gif(trs, target)
#    trs = mns ; target = 'mn' ; gif(trs, target)


## Grid Search

In [ ]:
# ==================== options ===================== #
cutoff = 0.85
config = "LR" ; log_true = True
target = lrs ; best_acc = ba
# ================================================== #


target_sort = np.argsort(target)

best_acc = best_acc[target_sort]
target_rng = target[target_sort]

first_cut = (np.where(best_acc>cutoff)[0][0])
last_cut = (np.where(best_acc>cutoff)[0][-1])

percent_50 = len(best_acc)//2
top_50 = np.argsort(best_acc)[-percent_50:]
top_50_min_idx = np.where(target_rng[top_50]==target_rng[top_50].min())[0][0]
top_50_max_idx = np.where(target_rng[top_50]==target_rng[top_50].max())[0][0]
top_50_min_target = target_rng[top_50][top_50_min_idx] ; top_50_min_acc = best_acc[top_50][top_50_min_idx]
top_50_max_target = target_rng[top_50][top_50_max_idx] ; top_50_max_acc = best_acc[top_50][top_50_max_idx]


best_sort = np.argsort(best_acc)
bests = best_sort[-5:]

plt.rcParams['figure.figsize'] = (60, 30)
plt.rcParams['xtick.labelsize']=50 
plt.rcParams['ytick.labelsize']=50 


fig, ax1 = plt.subplots()
if(log_true) : plt.gca().set_xscale('log')
#plt.plot(target_rng, best_acc)
plt.scatter(target_rng, best_acc, s=1000, c='k')
plt.scatter(target_rng[top_50], best_acc[top_50], s=1000, c='r')
plt.scatter(target_rng[bests], best_acc[bests], s=1000, c='g')


plt.ylim([best_acc.min()-5,best_acc.max()+5])

# Top 5 Hyp. values
plt.axvline(target_rng[bests[-1]], c='g', 
            label='Top1 : %.3f%% (%s %.3f)'%(best_acc[bests[-1]],config,target_rng[best_sort[-1]]))
plt.axvline(target_rng[bests[-2]], c='g', 
            label='Top2 : %.3f%% (%s %.3f)'%(best_acc[bests[-2]],config,target_rng[best_sort[-2]]))
plt.axvline(target_rng[bests[-3]], c='g', 
            label='Top3 : %.3f%% (%s %.3f)'%(best_acc[bests[-3]],config,target_rng[best_sort[-3]]))
plt.axvline(target_rng[bests[-4]], c='g', 
            label='Top4 : %.3f%% (%s %.3f)'%(best_acc[bests[-4]],config,target_rng[best_sort[-4]]))
plt.axvline(target_rng[bests[-5]], c='g', 
            label='Top5 : %.3f%% (%s %.3f)'%(best_acc[bests[-5]],config,target_rng[best_sort[-5]]))

loc = cutoff - 50

if(0):
    # First and Last cutoff acc.
    plt.axvline(target_rng[first_cut], c='r', ls='--')
    plt.text(target_rng[first_cut], loc, 'Cutoff'
             , c='r',fontsize=50, rotation=90 ,va='baseline'
             , ha='center', backgroundcolor='w')

    plt.axvline(target_rng[last_cut], c='r', ls='--')
    plt.text(target_rng[last_cut], loc, 'Cutoff'
             , c='r',fontsize=50, rotation=90 ,va='baseline'
             , ha='center', backgroundcolor='w')


    plt.axvline(top_50_min_target, c='r', ls='--')
    plt.text(top_50_min_target, loc, 'Top 50%'
             , c='r',fontsize=50, rotation=90 ,va='baseline'
             , ha='center', backgroundcolor='w')

    plt.axvline(top_50_max_target, c='r', ls='--')
    plt.text(top_50_max_target, loc, 'Top 50%'
             , c='r', fontsize=50, rotation=90 ,va='baseline'
             , ha='center', backgroundcolor='w')


plt.axhline(cutoff, c='k', ls='--')
plt.text(target_rng[10], cutoff, 'Threshold %.1f'%(cutoff), fontsize=50, va='baseline', ha='center', backgroundcolor='w')

plt.title('Accuracy for each configuration (%s)'%(config),fontsize=80) ## 타이틀 설정
plt.legend( prop={'size': 30})
plt.xticks(
            [top_50_min_target, 
            target_rng[first_cut], 
            target_rng[last_cut], 
            top_50_max_target], 
           
           [np.round(top_50_min_target,3), 
            np.round(target_rng[first_cut],3), 
            np.round(target_rng[last_cut],3), 
            np.round(top_50_max_target,3)],
            
            rotation=90
            )

plt.show()
plt.close()

In [ ]:
fig = plt.figure(figsize=(36,18)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

for i in range(na.shape[0]):
    ax.plot(na[i],marker='o') ## 선그래프 생성

#ax.legend() ## 범례 
#if(na.shape[1]==100):
#    ax.axvline(1) ; ax.axvline(3) ; ax.axvline(9) ; ax.axvline(27) ; ax.axvline(81)
plt.title('Total Learning Curves',fontsize=50) ## 타이틀 설정
plt.show()

In [ ]:
wds[idxs][-11]

In [ ]:
def labeling(topidx):
    if(is_grid): config_label = 'Top%d : %.3f%% (%s %.3f )'%(i+1,ba[topidx],'LR', lrs[topidx])
    else:
        if('cifar' in image_dir): 
            config_label = 'Top%d : %.3f%% (%s %.3f  %s %.5f  %s %.3f  )'%(
                i+1,ba[topidx], 'LR', lrs[topidx],'WD', wds[topidx], 'MN', 1-mns[topidx] )
        elif('uda' in image_dir) : 
            config_label = 'Top%d : %.3f%% (%s %.3f  %s %.5f  %s %.3f %s %.3f)'%(
                i+1,ba[topidx], 'LR', lrs[topidx],'WD', wds[topidx], 'MN', 1-mns[topidx], 'AdvW', advws[topidx] )
        elif('MDC' in image_dir) : 
            config_label = 'Top%d : %.3f%% (%s %.3f  %s %.5f  %s %.3f ACPR %d%d%d%d)'%(
                i+1,ba[topidx], 'LR', lrs[topidx],'WD', wds[topidx], 'MN', 1-mns[topidx], 
                arts[topidx], clips[topidx], prods[topidx], rws[topidx] )
    return config_label


top = 5

fig = plt.figure(figsize=(36,18)) ## 캔버스 생성
plt.rcParams['xtick.labelsize']=40
plt.rcParams['ytick.labelsize']=40

ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성
idxs = best_sort[-top:]
     
# pink background
for i in range(na.shape[0]):
    ax.plot(na[i],marker='o',color='#f3e5e1') #EFC2B8 #f3e5e1


colors = ['r','g','b','c','y']
if(top>5): colors = colors + ['k'] * (top-5) 

for i in range(top): 
    topidx = idxs[-(i+1)]
    config_label = labeling(topidx)
    ax.plot(na[topidx],marker='o', label= config_label, c=colors[i])
    
#ax.legend() ## 범례 
plt.title('Learning Curves : Top %d Sampling'%(top),fontsize=50) ## 타이틀 설정
plt.legend( prop={'size': 30}, loc='lower right')

#========================================== Ranking ==========================================#
xs = []
ratio = 0.4; steps = 10
for i in range(na.shape[1]):
    if((i+1)%steps==0):        
        rank = []
        temp_sort = np.argsort(na[:,i])
        na_i = na[:,i]
        remains = na_i[na_i!=0]
    
        [rank.insert(0, na.shape[0] - np.where(idx == temp_sort)[0][0] ) for idx in idxs]
        rank.append( sum(rank) / len(rank) )

        plt.text(i,ratio*i,str(rank[-1])+' / %d'%(len(remains)), fontsize=25, ha='center')
        plt.text(i,ratio*i-3,rank[:-1], fontsize=20, ha='center')

        ax.axvline(i, alpha=0.7)
        xs.append(i+1)
plt.xticks(xs)

plt.show()

In [ ]:
target_rng.shape

In [ ]:
# ========== options =========== #
top = 3
check_epoch = 27
# ============================= #

target = lrs
target_sort = np.argsort(target)
best_acc = ba
best_acc = best_acc[target_sort]
target_rng = target[target_sort]

idxs = best_sort[-top:]
acc_chk=na[:,check_epoch]
sort_chk=np.argsort(acc_chk)
idxs_chk=sort_chk[-top:]

fig = plt.figure(figsize=(36,18)) ## 캔버스 생성
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

for i in range(na.shape[0]):
    ax.plot(na[i],marker='o',color='#f3e5e1') ## 선그래프 생성
    
colors = ['r','g','b']
for i in range(top): 
    topidx = idxs[-(i+1)]
    ax.plot(na[topidx],marker='o', 
            label='Top%d : %.3f%% (%s %.3f)'%(i+1,ba[topidx],config, target_rng[topidx]),
            c=colors[i]) ## 선그래프 생성
    
colors = ['c','m','k']
for i in range(top): 
    topidx = idxs_chk[-(i+1)]
    ax.plot(na[topidx],marker='o', 
            label='(ep%d)Top%d : %.3f%% (%s %.3f)'%(check_epoch,i+1,ba[topidx],config, target_rng[topidx]),
            c=colors[i]) ## 선그래프 생성
    
ax.axvline(check_epoch)
plt.title('Learning Curves : Top %d Compares (epoch %d vs max)'%(top,check_epoch),fontsize=50) ## 타이틀 설정
plt.legend( prop={'size': 30})
plt.show()

## Ranking

In [ ]:
cutoff = False

class VisualizeLC():
    def __init__(self, objective_val, config_name, all_curves, filepath):
        analysis, df_analysis, best_config, best_accuracy = fn_tune.load_tune_analysis(filepath)
        best_curve = fn_tune.get_best_trial_curve(df_analysis, val=objective_val)
#        all_curves = fn_tune.get_trial_curves(df_analysis, val=objective_val)
        trial_order = fn_tune.get_trial_order(df_analysis)
        # Dataset
        self.df = df_analysis
        self.bc = best_config
        self.ba = best_accuracy
        self.config_name = config_name
        self.objective_val = objective_val
        self.n_trial = np.shape(self.df)[0]
        # Learning Curves
        self.best_curve = best_curve
        self.all_curves = all_curves
        self.trial_order = trial_order
        # Trial Scheduler: ASHA
        self._max_t = self.df["training_iteration"].max()
        self._min_t = 1
        self._eta = 3
        self._s = 0
        self.n_rungs = int(np.log(self._max_t / self._min_t) / np.log(self._eta) - self._s + 1)
        self.rungs = [self._min_t * self._eta ** (k + self._s) for k in range(self.n_rungs)]
        
    def _get_best_objval(self):
        if self.objective_val == "accuracy":
            best_val = self.df["best_acc"].max()
        if self.objective_val == "loss":
            best_val = np.min(self.all_curves)
        return best_val
        
    def plot_best_curve(self):
        epochs = np.arange(len(self.best_curve))
        lbl = "Best {}: {:.3f}\nBest Config: {:.3f}".format(
            self.objective_val.capitalize(),
            self._get_best_objval(),
            self.bc[self.config_name])
        plt.plot(epochs, self.best_curve, label=lbl)
        plt.xlabel('Epochs')
        plt.ylabel('Validation {}'.format(self.objective_val.capitalize()))
        plt.title('Best Trial')
        if self.objective_val == "accuracy":
            plt.legend(fontsize=12, loc="lower right")
        else:
            plt.legend(fontsize=12, loc="upper right")
        plt.grid()
        plt.show()
        return None
    
    def plot_entire_curves(self):
        epochs = np.arange(np.shape(self.all_curves)[1])
        vmin = np.min(self.all_curves)
        vmax = np.max(self.all_curves)
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 7))
        # Mark Periods where Pruning Occurs
        for r in self.rungs:
            plt.vlines(r, vmin, vmax, color='k', linestyles='dashed', alpha=0.5)
        # Plot Curves
        if self.objective_val == "accuracy":
            best_acc = 0
            for i in range(len(self.all_curves)):
                curr_acc = max(self.all_curves[i])
                if curr_acc > best_acc:
                    best_acc = curr_acc
                    best_idx = i
                lbl = '{}={}: {:.2f}%'.format(self.config_name.upper(), self.df["config/"+self.config_name][self.trial_order[i]], curr_acc)
                ax.plot(self.all_curves[i], label=lbl)
            rank1 = np.argmax(np.array(self.all_curves), axis=0) == best_idx # points at which the best trial becomes best
            ax.scatter(epochs[rank1], np.ones(len(epochs),)[rank1]*np.ceil(vmin), color='tab:green')
        else:
            best_loss = 100
            for i in range(len(self.all_curves)):
                curr_loss = min(self.all_curves[i])
                if curr_loss < best_loss:
                    best_loss = curr_loss
                    best_idx = i
                lbl = '{}={}: {:.3f}%'.format(self.config_name.upper(), self.df["config/"+self.config_name][self.trial_order[i]], curr_loss)
                ax.plot(self.all_curves[i], label=lbl)
            rank1 = np.argmin(np.array(self.all_curves), axis=0) == best_idx
            ax.scatter(epochs[rank1], np.ones(len(epochs),)[rank1]*np.floor(vmax), color='tab:green')
        # Set Legend
        if len(self.all_curves) <= 15:
            leg = plt.legend(fontsize=11, loc="center right")
            leg.get_texts()[best_idx].set_color('green')
        # Set Labels
        plt.xlabel('Epochs')
        plt.ylabel('Validation {}'.format(self.objective_val.capitalize()))
        plt.title('All Trials')
        plt.grid()
        plt.show()
        return None
    
    def _top_k(self, k):
        if self.objective_val == "accuracy":
            metric = -self.df["best_acc"]
        if self.objective_val == "loss":
            metric = self.df["loss"] # final loss (not best)
        assert k <= self.n_trial
        top_idx = np.argsort(metric)[:k]
        return top_idx
    
    def _rank_trials(self):
        data = -np.array(self.all_curves)
        ranks = data.argsort(axis=0).argsort(axis=0)
        # alternatively: use `stats.rankdata(np.array(self.all_curves), method='min', axis=0)` to consider tied values
        return ranks
    
    def get_ordered_trial_id(self, k):
        top_idx = self._top_k(k)
        top_idx_ord = [self.trial_order.index(i) for i in top_idx] # indices chronologically ordered by trials
        return top_idx_ord
    
    def rank_trials_pctl(self, p, epoch_dc):
        if epoch_dc is None:
            epoch_dc = np.arange(9, 100, 10) # epochs at which the trial ranks will be computed
        if 99 not in epoch_dc:
            epoch_dc = np.insert(epoch_dc, len(epoch_dc), self._max_t - 1)
            warnings.warn("Total epoch should be included in `epoch_dc`. Added automatically.", Warning)
        data = np.array(self.all_curves)

        rank_results = dict() # best k trials at each epoch
        rank_overlap = [] # fraction of best k trials at each epoch against overall best trials
        rank_consistency = [] # fraction of how many top 5 trials are included in the best k trials at each epoch
        for i in epoch_dc:
            crop_curves = data[:, :(i+1)] # curves cut at specific epochs
            if self.objective_val == "accuracy":
                metric = -np.max(crop_curves, axis=1)
            if self.objective_val == "loss":
                metric = np.min(crop_curves, axis=1)
                
            # Trial Cutoff
            if(cutoff): k = int(np.round(p * 256))
            else: k = int(np.round(p * self.n_trial))


            top_idx = np.argsort(metric)[:k]
            rank_results.update({str(i): top_idx})
            if i == epoch_dc[-1]:
                top5 = np.argsort(metric)[:5]
        for i in range(len(epoch_dc)):
            ovp = np.intersect1d(rank_results[str(epoch_dc[i])], rank_results[str(self._max_t - 1)])
            rank_overlap.append('{}/{}'.format(len(ovp), k))
            cst = np.intersect1d(top5, rank_results[str(epoch_dc[i])])
            rank_consistency.append('{}/5'.format(len(cst)))
        return rank_results, rank_overlap, rank_consistency, epoch_dc
    
    def rank_trials_pctl_prune(self, p, epoch_dc):
        # Configure Input Parameters
        if epoch_dc is None:
            epoch_dc = np.arange(9, 100, 10) # epochs at which the trial ranks will be computed
        if 99 not in epoch_dc:
            epoch_dc = np.insert(epoch_dc, len(epoch_dc), self._max_t - 1)
            warnings.warn("Total epoch should be included in `epoch_dc`. Added automatically.", Warning)
        # Get Data and Preallocate Outputs
        data = np.array(self.all_curves)
        rank_results = dict() # best k trials at each epoch
        rank_atfinal = dict() # best k tirals at the end epoch
        rank_overlap = [] # fraction of best k trials at each epoch against overall best trials
        rank_consistency = [] # fraction of how many top 5 trials are included in the best k trials at each epoch
        threshold = [] # cutoff for pruning (i.e., how many trials will survive)
        # Extract Rankings at Each Particular Epoch
        
        # Trial Cutoff
        if(cutoff): k = int(np.round(p * 256))
        else: k = int(np.round(p * self.n_trial))


        for n, i in enumerate(epoch_dc):
            crop_curves = data[:, :(i+1)] # curves cut at specific epochs
            if self.objective_val == "accuracy":
                metric = -np.max(crop_curves, axis=1)
            if self.objective_val == "loss":
                metric = np.min(crop_curves, axis=1)
            if n > 0:
                
                # Trial Cutoff
                if(cutoff):bottom_idx = np.setdiff1d(np.arange(256), top_idx)
                else: bottom_idx = np.setdiff1d(np.arange(self.n_trial), top_idx)

                if self.objective_val == "accuracy":
                    metric[bottom_idx] = 0 # assign 0 to pruned trials
                if self.objective_val == "loss":
                    metric[bottom_idx] = np.Inf # assign positive infinity to pruned trials
            top_idx = np.argsort(metric)[:k]
            rank_results.update({str(i): top_idx})
            threshold.append(k)
            k = int(np.round(p * k)) # update k
            if k == 0: k = 1
        # Extract Rankings at Total Epoch
        if self.objective_val == "accuracy":
            metric = -np.max(data, axis=1)
        if self.objective_val == "loss":
            metric = np.min(data, axis=1)
        for i in range(len(epoch_dc)):
            top_idx = np.argsort(metric)[:threshold[i]]
            rank_atfinal.update({str(epoch_dc[i]): top_idx})
        # Compute Top k Trials at Total Epoch
        metric = -np.max(data, axis=1)
        topk = [np.argsort(metric)[:j] for j in np.arange(1, 6, 1)]
        # Compute Rank Overlap and Consistency
        for i in range(len(epoch_dc)):
            ovp = np.intersect1d(rank_results[str(epoch_dc[i])], rank_atfinal[str(epoch_dc[i])])
            rank_overlap.append('{}/{}'.format(len(ovp), threshold[i]))
            if threshold[i] >= 5:
                cst = np.intersect1d(topk[-1], rank_results[str(epoch_dc[i])])
                rank_consistency.append('{}/{}'.format(len(cst), len(topk[-1])))
            else:
                cst = np.intersect1d(topk[threshold[i]-(5+1)], rank_results[str(epoch_dc[i])])
                rank_consistency.append('{}/{}'.format(len(cst), len(topk[threshold[i]-(5+1)])))
        return rank_results, rank_overlap, rank_consistency, epoch_dc

    
    def plot_top_trials(self, k):
        top_idx = self._top_k(k)
        top_idx_ord = [self.trial_order.index(i) for i in top_idx]
        rst_idx_ord = np.delete(np.arange(self.n_trial), top_idx_ord)
        epochs = np.arange(np.shape(self.all_curves)[1])
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 7))
        # Mark Periods where Pruning Occurs
        vmin = np.min(self.all_curves)
        vmax = np.max(self.all_curves)
        for r in self.rungs:
            plt.vlines(r, vmin, vmax, color='k', linestyles='dashed', alpha=0.5)
        # Plot Rest of the Trials
        for idx in rst_idx_ord:
            plt.plot(epochs, self.all_curves[idx], color='#dbb2b2', marker='o', markersize=3, alpha=0.2)
        # Plot Trials with Best Objective Values
        for i, idx in enumerate(top_idx_ord):
            if self.objective_val == "accuracy":
                metric_val = max(self.all_curves[idx])
                leg_loc = "lower right"
            else:
                metric_val = min(self.all_curves[idx])
                leg_loc = "upper right"
            lbl = "Best #{:d} ({:.3f}): {}={:.3f}".format(i, metric_val, self.config_name.upper(), self.df["config/"+self.config_name][self.trial_order[idx]])
            # alternatively, use `top_idx[i]` instead of `self.trial_order[idx]`
            plt.plot(epochs, self.all_curves[idx], marker='o', markersize=3, lw=1.5, label=lbl)
        plt.xlabel('Epochs')
        plt.ylabel(f"Validation {self.objective_val.capitalize()}")
        plt.title('Learning Curves: Top {} Trials'.format(k))
        plt.legend(loc=leg_loc, fontsize=11)
        plt.show()
        return None
    
    def plot_top_trials_partial(self, k, epoch_criterion):
        top_idx = self._top_k(k)
        top_idx_ord = [self.trial_order.index(i) for i in top_idx]
        if self.objective_val == "accuracy":
            prt_idx_ord = np.argsort(-np.max(np.array(self.all_curves)[:,:epoch_criterion], axis=1))[:k]
            leg_loc = 'lower right'
        if self.objective_val == "loss":
            prt_idx_ord = np.argsort(np.min(np.array(self.all_curves)[:,:epoch_crtierion], axis=1))[:k]
            leg_loc = 'center right'
        prt_idx_ord = prt_idx_ord.tolist()
        rst_idx_ord = np.delete(np.arange(self.n_trial), np.unique(top_idx_ord + prt_idx_ord))
        epochs = np.arange(np.shape(self.all_curves)[1])
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 7))
        # Mark Periods where Pruning Occurs
        vmin = np.min(self.all_curves)
        vmax = np.max(self.all_curves)
        for r in self.rungs:
            plt.vlines(r, vmin, vmax, color='k', linestyles='dashed', alpha=0.5)
        # Mark Criterion
        plt.vlines(epoch_criterion, vmin, vmax, color='tab:blue', linestyles='solid', label='Partial LC cutoff')
        # Plot Rest of the Trials
        for idx in rst_idx_ord:
            plt.plot(epochs, self.all_curves[idx], color='#dbb2b2', marker='o', markersize=3, alpha=0.2)
        # Plot Trials with Best Objective Values Overall
        lbl = None
        for i, idx in enumerate(top_idx_ord):
            if self.objective_val == "accuracy":
                metric_val = max(self.all_curves[idx])
            else:
                metric_val = min(self.all_curves[idx])
            lbl = "Best #{:d} ({:.3f}): {}={:.3f}".format(i, metric_val, self.config_name.upper(), self.df["config/"+self.config_name][self.trial_order[idx]])
            plt.plot(epochs, self.all_curves[idx], marker='o', markersize=3, lw=1.5, label=lbl)
        # Plot Trials with Best Objective Values at Particular Epoch
        lbl = None
        cmap = clrs.LinearSegmentedColormap.from_list('custom', ['#11498E', '#3A73C4', '#69A0C3', '#ABC08E', '#C7DF2F'], N=k).reversed()
        for i, idx in enumerate(prt_idx_ord):
            if self.objective_val == "accuracy":
                metric_val = max(self.all_curves[idx])
            else:
                metric_val = min(self.all_curves[idx])
            lbl = "Partial #{:d} ({:.3f}): {}={:.3f}".format(i, metric_val, self.config_name.upper(), self.df["config/"+self.config_name][self.trial_order[idx]])
            plt.plot(epochs, self.all_curves[idx], color=cmap(i), marker='o', markersize=3, lw=1.5, label=lbl)
        plt.xlabel('Epochs')
        plt.ylabel(f"Validation {self.objective_val.capitalize()}")
        plt.title('Learning Curves: Top {} Trials at {} and {} epochs'.format(k, epoch_criterion, self._max_t-1))
        plt.legend(loc=leg_loc, fontsize=11, bbox_to_anchor=(1.28, 0))
        plt.show()
        return None
    
    def plot_best_per_config(self, epoch=None, top_pct=1):
        config_arr = np.array(self.df["config/"+self.config_name][self.trial_order])
        if epoch is None:
            epoch = self._max_t - 1
        rung_curves = [curve[:epoch+1] for curve in self.all_curves]
        if self.objective_val == "accuracy":
            metric_arr = np.array([-1 * max(curve) for curve in rung_curves])
            flag = -1
            metric_name = "best_acc"
        elif self.objective_val == "loss":
            metric_arr = np.array([min(curve) for curve in rung_curves])
            flag = 1
            metric_name = "loss"
        rung_rank = np.argsort(metric_arr)
        metric_arr = flag * metric_arr
        vmin, vmax = min(metric_arr), max(metric_arr)
        if self.objective_val == "accuracy":
            thr = int(vmax) - 1
            crossing_idx = np.where(metric_arr >= thr)[0]
            crossing_cfg = np.array([config_arr[idx] for idx in crossing_idx])
            crossing = [np.where(config_arr == min(crossing_cfg))[0][0], np.where(config_arr == max(crossing_cfg))[0][0]]
        elif self.objective_val == "loss":
            thr = int(vmin) + 1
            crossing_idx = np.where(metric_arr <= thr)[0]
            crossing_cfg = np.array([config_arr[idx] for idx in crossing_idx])
            crossing = [np.where(config_arr == min(crossing_cfg))[0][0], np.where(config_arr == max(crossing_cfg))[0][0]]
        clr_arr = np.array(["k"] * self.n_trial)
        rung_top5 = list(map(int, np.argsort(flag * self.df[metric_name][self.trial_order].to_numpy())[:5])) # overall top 5 trial indices
        if top_pct < 1:
            rung_topk = list(map(int, rung_rank[:int(self.n_trial * top_pct)]))
            clr_arr[rung_topk] = "r"
        clr_arr[rung_top5] = "g"
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 7))
        ax.scatter(config_arr, metric_arr, lw=1.5, color=clr_arr, marker='o', zorder=1)
        for i in range(2):
            ax.text(config_arr[crossing[i]], (vmin + vmax)/2, '{:.2f}% {}={:.3f}'.format(metric_arr[crossing[i]], self.config_name.upper(), config_arr[crossing[i]]), 
                    rotation=270, verticalalignment='center')
        ax.hlines(thr, min(config_arr), max(config_arr), linestyles='dashed', alpha=0.5, label='Threshold', zorder=2)
        ax.vlines([config_arr[crossing[0]], config_arr[crossing[1]]], vmin, vmax, linestyles='dashed', alpha=0.5, color='tab:red', label='Crossings', zorder=2)
        ax.set_xscale('log')
        ax.set_xlabel(self.config_name.upper() + ' Configurations')
        ax.set_ylabel(f"Validation {self.objective_val.capitalize()}")
        ax.set_title(f"Best Objective Values Per Each Configuration\n(Top {top_pct*100}%, Reference Epoch: {epoch}, Threshold: {thr:.2f})")
        plt.legend(loc='best')
        plt.show()
        return None
    
    def _find_str_idx(self, s, ch):
        return [i for i, letter in enumerate(s) if letter == ch]
    
    def plot_rank_bars(self, top_pnt, epoch_dc, prune=False, verbose=True):
        if prune:
            rank_res, rank_ovp, rank_cst, epoch_dc = self.rank_trials_pctl_prune(top_pnt, epoch_dc)
            title = 'Summary of Ranks at Particular Epochs\nwith Pruning Considered : %.4f'%(top_pnt)
        else:
            rank_res, rank_ovp, rank_cst, epoch_dc = self.rank_trials_pctl(top_pnt, epoch_dc)
            title = 'Summary of Ranks at Particular Epochs : %.4f'%(top_pnt)
        idx_ovp = [self._find_str_idx(ovp, '/')[0] for ovp in rank_ovp]
        idx_cst = [self._find_str_idx(cst, '/')[0] for cst in rank_cst]
        overlap = [int(rank_ovp[i][:idx_ovp[i]]) for i in range(len(idx_ovp))]
        consistency = [int(rank_cst[i][:idx_cst[i]]) for i in range(len(idx_cst))]
        # Plot Bar Graphs
        vmin = 0
        vmax = np.round(top_pnt*self.n_trial, -1) + self.n_trial*0.10
        plt.figure()
        
        
        plt.rcParams['figure.figsize'] = (25,25)
        plt.rcParams['xtick.labelsize']= 30
        plt.rcParams['ytick.labelsize']= 30

        
        
        bar_ovp = plt.bar(np.arange(len(overlap)), overlap, width=0.5, label='Rank Overlap')
        bar_cst = plt.bar(np.arange(len(consistency)), consistency, width=0.5, label='Rank Consistency')
        for i, bar in enumerate(bar_ovp.patches):
            y_annot = bar.get_height()
            if overlap[i] == 0 or overlap[i] == consistency[i] or np.abs(overlap[i]-consistency[i]) < (vmax-vmin)*0.1: 
                scaler = 0.1
                if np.abs(overlap[i]-consistency[i]) < 5: 
                    scaler = 0.07
                y_annot = y_annot + (np.round(top_pnt*100, -1) + 10) *  scaler # add x% of the y axis range
            plt.annotate(rank_ovp[i],
                         (bar.get_x() + bar.get_width() / 2, y_annot),
                         ha='center', va='bottom',
                         size=24, fontweight='bold')
        for i, bar in enumerate(bar_cst.patches):
            txt_clr = 'w'
            if consistency[i] == 0 or consistency[i] == overlap[i] or np.abs(overlap[i]-consistency[i]) < (vmax-vmin)*0.1: 
                txt_clr = 'k'
            plt.annotate(rank_cst[i],
                         (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                         ha='center', va='bottom',
                         size=24, fontweight='bold', color=txt_clr)
        plt.xlabel('Epoch', size=30)
        plt.xticks(ticks=np.arange(len(epoch_dc)), labels=[str(epoch) for epoch in epoch_dc])
        plt.ylabel('Fractions', size=30)
        plt.ylim((vmin, vmax))
        plt.title(title, fontsize=30)
        plt.legend(loc='upper right', fontsize=20)
        plt.show()
        # Print Plot Summary
        if verbose:
            print(f'Rank at particular epoch (Top ~{top_pnt*100:.1f}%): \n', rank_res)
            print('Overlap between best trials at particular epoch and the end: \n', rank_ovp)
            print('Fraction of best 5 trials in each particular epoch: \n', rank_cst)
        return None

    def rank_trials_pctl_prune_seq(self, p, epoch_dc):
        # Configure Input Parameters
        if epoch_dc is None:
            epoch_dc = np.arange(9, 100, 10) # epochs at which the trial ranks will be computed
        if 99 not in epoch_dc:
            epoch_dc = np.insert(epoch_dc, len(epoch_dc), self._max_t - 1)
            warnings.warn("Total epoch should be included in `epoch_dc`. Added automatically.", Warning)
        # Get Data and Preallocate Outputs
        data = np.array(self.all_curves)
        rank_results = dict() # best k trials at each epoch
        rank_atfinal = dict() # best k tirals at the end epoch
        rank_overlap = [] # fraction of best k trials at each epoch against overall best trials
        rank_consistency = [] # fraction of how many top 5 trials are included in the best k trials at each epoch
        threshold = [] # cutoff for pruning (i.e., how many trials will survive)
        # Extract Rankings at Each Particular Epoch
        k = int(np.round(p * self.n_trial))
        for n, i in enumerate(epoch_dc):
            crop_curves = data[:, :(i+1)] # curves cut at specific epochs
            if self.objective_val == "accuracy":
                metric = -np.max(crop_curves, axis=1)
            if self.objective_val == "loss":
                metric = np.min(crop_curves, axis=1)
            if n > 0:
                bottom_idx = np.setdiff1d(np.arange(self.n_trial), top_idx_first)
                if self.objective_val == "accuracy":
                    metric[bottom_idx] = 0 # assign 0 to pruned trials
                if self.objective_val == "loss":
                    metric[bottom_idx] = np.Inf # assign positive infinity to pruned trials
            top_idx = np.argsort(metric)[:k]
            rank_results.update({str(i): top_idx})
            threshold.append(k)
            if(n==0):
                k = int(np.round(p * k)) # update k
                top_idx_first = top_idx

#            if k == 0: k = 1

        # Extract Rankings at Total Epoch
        if self.objective_val == "accuracy":
            metric = -np.max(data, axis=1)
        if self.objective_val == "loss":
            metric = np.min(data, axis=1)
        for i in range(len(epoch_dc)):
            top_idx = np.argsort(metric)[:threshold[i]]
            rank_atfinal.update({str(epoch_dc[i]): top_idx})
        # Compute Top k Trials at Total Epoch
        metric = -np.max(data, axis=1)
        topk = [np.argsort(metric)[:j] for j in np.arange(1, 6, 1)]
        # Compute Rank Overlap and Consistency
        for i in range(len(epoch_dc)):
            ovp = np.intersect1d(rank_results[str(epoch_dc[i])], rank_atfinal[str(epoch_dc[i])])
            rank_overlap.append('{}/{}'.format(len(ovp), threshold[i]))
            if threshold[i] >= 5:
                cst = np.intersect1d(topk[-1], rank_results[str(epoch_dc[i])])
                rank_consistency.append('{}/{}'.format(len(cst), len(topk[-1])))
            else:
                cst = np.intersect1d(topk[threshold[i]-(5+1)], rank_results[str(epoch_dc[i])])
                rank_consistency.append('{}/{}'.format(len(cst), len(topk[threshold[i]-(5+1)])))
        return rank_results, rank_overlap, rank_consistency, epoch_dc
    
    def plot_rank_bars_seq(self, top_pnt, epoch_dc, prune=False, verbose=True):
        rank_res, rank_ovp, rank_cst, epoch_dc = self.rank_trials_pctl_prune_seq(top_pnt, epoch_dc)
        title = 'Summary of Ranks at Particular Epochs\nwith Pruning Considered'
        idx_ovp = [self._find_str_idx(ovp, '/')[0] for ovp in rank_ovp]
        idx_cst = [self._find_str_idx(cst, '/')[0] for cst in rank_cst]
        overlap = [int(rank_ovp[i][:idx_ovp[i]]) for i in range(len(idx_ovp))]
        consistency = [int(rank_cst[i][:idx_cst[i]]) for i in range(len(idx_cst))]
        # Plot Bar Graphs
        vmin = 0
        vmax = np.round(top_pnt*self.n_trial, -1) + self.n_trial*0.10
        plt.figure()
        
        
        plt.rcParams['figure.figsize'] = (25,25)
        plt.rcParams['xtick.labelsize']= 30
        plt.rcParams['ytick.labelsize']= 30

        
        
        bar_ovp = plt.bar(np.arange(len(overlap)), overlap, width=0.5, label='Rank Overlap')
        bar_cst = plt.bar(np.arange(len(consistency)), consistency, width=0.5, label='Rank Consistency')
        for i, bar in enumerate(bar_ovp.patches):
            y_annot = bar.get_height()
            if overlap[i] == 0 or overlap[i] == consistency[i] or np.abs(overlap[i]-consistency[i]) < (vmax-vmin)*0.1: 
                scaler = 0.1
                if np.abs(overlap[i]-consistency[i]) < 5: 
                    scaler = 0.07
                y_annot = y_annot + (np.round(top_pnt*100, -1) + 10) *  scaler # add x% of the y axis range
            plt.annotate(rank_ovp[i],
                         (bar.get_x() + bar.get_width() / 2, y_annot),
                         ha='center', va='bottom',
                         size=24, fontweight='bold')
        for i, bar in enumerate(bar_cst.patches):
            txt_clr = 'w'
            if consistency[i] == 0 or consistency[i] == overlap[i] or np.abs(overlap[i]-consistency[i]) < (vmax-vmin)*0.1: 
                txt_clr = 'k'
            plt.annotate(rank_cst[i],
                         (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                         ha='center', va='bottom',
                         size=24, fontweight='bold', color=txt_clr)
        plt.xlabel('Epoch', size=30)
        plt.xticks(ticks=np.arange(len(epoch_dc)), labels=[str(epoch) for epoch in epoch_dc])
        plt.ylabel('Fractions', size=30)
        plt.ylim((vmin, vmax))
        plt.title(title, fontsize=30)
        plt.legend(loc='upper right', fontsize=20)
        plt.show()
        # Print Plot Summary
        if verbose:
            print(f'Rank at particular epoch (Top ~{top_pnt*100:.1f}%): \n', rank_res)
            print('Overlap between best trials at particular epoch and the end: \n', rank_ovp)
            print('Fraction of best 5 trials in each particular epoch: \n', rank_cst)
        return None

In [ ]:
from ray import tune
import torch
import torchvision
from torch import optim
from utils import fn_tune
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import mpl_toolkits.mplot3d
import numpy as np
import scipy.stats as stats
import seaborn as sns
import warnings

objective_val = "accuracy"
config_name = "lr"
file_path = image_dir

test = VisualizeLC(objective_val, config_name,na, file_path)
#test.plot_best_curve()
#test.plot_entire_curves()
#test.plot_top_trials(5)
#ids = test.get_ordered_trial_id(k=5)
#test.plot_trial_ranking(ids)
#test.plot_top_trials_partial(5, 49)
#test.plot_best_per_config(99, 0.5)

#if('cifar' in checkp):  epoch_dc = np.array([1,3,9, 19, 29, 39, 49, 59, 69, 79, 89, 99])
#if('cifar' in checkp):  epoch_dc = np.array([49,99])

#if('cifar' in checkp):  epoch_dc = np.array([1,3,9, 27, 81, 99])
#if('cifar' in checkp):  epoch_dc = np.array([3,9, 27, 81, 99])
#if('cifar' in checkp):  epoch_dc = np.array([24,49,74,99])
#if('cifar' in checkp):  epoch_dc = np.array([9,19,29,39,49,59,69,79,89,99, 109, 119, 124])
if('cifar' in checkp):  epoch_dc = np.array([9,19,29,39,49])

#elif('uda' in checkp) : epoch_dc = np.array([1,3,9, 19, 29])
elif('uda' in checkp) : epoch_dc = np.array([4,9,14,19,24,29])

#elif('MDC' in checkp):  epoch_dc = np.array([1,3,9, 19, 29])
elif('MDC' in checkp):  epoch_dc = np.array([4,9,14, 19,24, 29])
    
if('FS' in checkp): epoch_dc = np.array([1,3,9, 19, 29, 39, 49])
if('FS' in checkp): epoch_dc = np.array([4,9,14,19,24,29,34,39,44,49])

epoch_dc = np.array([50,100,150,200,250,300,350,400,450,500,550,600,650,700,749])

top_pnt = 1/2
test.plot_rank_bars(top_pnt, epoch_dc, prune=False, verbose=False)
#test.plot_rank_bars_seq(top_pnt, epoch_dc, prune=True, verbose=True)
#test.plot_rank_bars(top_pnt, epoch_dc, prune=True, verbose=True)

top_pnt = 1/4
test.plot_rank_bars(top_pnt, epoch_dc, prune=False, verbose=False)


In [ ]:
top_pnt = 1/2
test.plot_rank_bars(top_pnt, epoch_dc, prune=True, verbose=False)
#test.plot_rank_bars_seq(top_pnt, epoch_dc, prune=True, verbose=True)
#test.plot_rank_bars(top_pnt, epoch_dc, prune=True, verbose=True)

top_pnt = 1/4
test.plot_rank_bars(top_pnt, epoch_dc, prune=True, verbose=False)


